<a href="https://colab.research.google.com/github/wzk1015/video-bgm-generation/blob/develop/CMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demo of Controllable Music Transformer**

We provide a colab notebook for running inference with CMT. You can upload a video and generate a background music using this notebook.

# 1. Clone the repo and download checkpoint from google drive

In [ ]:
import os
from google.colab import files
import json
# get flownet2-pytorch source
os.chdir('/content')
!git clone https://github.com/wzk1015/video-bgm-generation -b develop
os.chdir('/content/video-bgm-generation')

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
# gdd.download_file_from_google_drive(file_id='1Ud2-GXEr4PbRDDe-FZJwzqqZrbbWFxM',dest_path='exp/loss_8_params.pt', showsize=True)
gdd.download_file_from_google_drive(file_id='1VZkoiVOonffpJWxZah-AdQkxaTFIzZ6q',dest_path='./Essential Keys-sforzando-v9.6.sf2', showsize=True)

In [ ]:
# DEBUG CELL, IGNORE THIS

# !cp /content/drive/MyDrive/CMT/loss_8_params.pt exp/
# !cp /content/drive/MyDrive/video-bgm-generation/videos/final_640.mp4 videos/test.mp4

# !pip install midi2audio
# !pip install pyfluidsynth

# !apt-get install fluidsynth

# # !fluidsynth
# import midi2audio
# fs = midi2audio.FluidSynth('./splendid 136.sf2')
# fs.midi_to_audio('get_2.mid', 'output.mp3')

# os.chdir("/content/video-bgm-generation/videos/")
# !mv test.mp4 test_raw.mp4
!ls -lh "/content/video-bgm-generation/inference/"

# 2. Install dependencies

In [ ]:
!pip install -r py3_requirements.txt
# this may take a long time

In [ ]:
os.chdir("/content/video-bgm-generation/src/video2npz/visbeat3/")
!python setup.py install

# 3. Upload your video

It is recommended to use videos **less than 2 minutes**, otherwise it gets really slow

In [ ]:
os.chdir("/content/video-bgm-generation/")
uploaded = files.upload()
assert len(uploaded) == 1, "upload one video file only"
filename = list(uploaded.keys())[0]
os.system(f'mv {filename} videos/test_raw.mp4')

Convert to 360p to speed up extracting optical flow and visbeats

In [ ]:
os.chdir("/content/video-bgm-generation/videos/")
!rm test.mp4
!ffmpeg -i test_raw.mp4 -strict -2 -vf scale=-1:360 test.mp4

# 4. Convert video into npz file

In [ ]:
os.chdir("/content/video-bgm-generation/src/video2npz/")
!rm -r VisBeatAssets/ fig/ flow/ image/ optical_flow/
!bash video2npz.sh ../../videos/test.mp4
os.chdir("../")
# extracting optical flow and visbeats may be slow

# 5. Run the model to generate music in MIDI format (.mid) 

In [ ]:
os.chdir("/content/video-bgm-generation/src/")
!python gen_midi_conditional.py -f "../inference/test.npz" -c "../exp/loss_8_params.pt"

# 6. Convert midi into audio: use **GarageBand (recommended)** or midi2audio

Remember to **set tempo to the value of tempo in video2npz/metadata.json**

In [ ]:
os.chdir("/content/video-bgm-generation/src/")
files.download('../inference/test.npz_0.mid')

with open("video2npz/metadata.json") as f:
    tempo = json.load(f)['tempo']
    print("tempo:", tempo)

### Generate audio with midi2audio

Instead of running this cell, we strongly recommend using GarageBand or other softwares, since their soundfonts are better. But this also works fine

In [ ]:
# TODO

# 7. Combine original video and audio into video with BGM
Generate/upload the audio file under `inference`, name it as `music.mp3`, and run ffmpeg to combine video and music

In [ ]:
os.chdir("/content/video-bgm-generation/inference/")
!rm output.mp4
!ffmpeg -i ../videos/test.mp4 -i music.mp3 -c:v copy -c:a aac -strict experimental -map 0:v:0 -map 1:a:0 output.mp4
files.download('output.mp4')